In [2]:
## Import necessary modules
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import date2num, AutoDateFormatter, AutoDateLocator, WeekdayLocator, MonthLocator, DayLocator, DateLocator, DateFormatter
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
from matplotlib.ticker import AutoMinorLocator
import numpy as np
import datetime, calendar
#from datetime import datetime
import matplotlib.patches as mpatches
%matplotlib inline

## Import data frome pickle generated from muß ein file mit agentenstats sein
data = pd.read_pickle('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/pkl_plots_calls_1458.pkl')
data2=data.drop(['dt','dd','hh','tt','acw','wd','yy','ag','ht','vl','mm','an'],1)
wsplit=data2.groupby(['ww','lo','bz']).sum()

ValueError: labels ['dt' 'dd' 'hh' 'tt' 'acw' 'wd' 'yy' 'ag' 'ht' 'vl' 'mm' 'an'] not contained in axis

In [ ]:
xlframe=pd.read_excel('/home/keuch/gits/keuch/code_box/pyt/spreadsheetparsing/test_stats/archiv/CE_alle_Agenten_taeglich_2017-03-27.xls')
xlframe

In [ ]:
unstckd=wsplit.unstack().unstack()
kernzeit=unstckd['be','k']
nebenzeit=unstckd['be','n']

fig=plt.figure(figsize=(12,6))
fig.suptitle('Anrufverteilung Hagenow zu Berlin')
kzh="#003873"
kzb="#EE0042"
nzh="#899EB2"
nzb="#C7798F"

ax=fig.add_subplot(131)
ax.margins(0,0)
ax.set_facecolor('#FFFAE4')
ax.set_xticks(unstckd.index)
ax.set_title('Gesamte Zeit')
ax.set_ylabel('angenommene Calls')
ax.set_xlabel('Kalenderwoche')
ax.yaxis.grid(True, which='major', color='#8C8C8C', linestyle=':')
ax.set_axisbelow(True)
p2 = ax.bar(unstckd.index, nebenzeit['B']+kernzeit['B'], width=.8, color=kzb, bottom=nebenzeit['H']+kernzeit['H'], label="Ber")
p1 = ax.bar(unstckd.index, nebenzeit['H']+kernzeit['H'], width=.8, color=kzh, label="Hag")
maxcalls=ax.get_ylim()

ax2=fig.add_subplot(132)
ax2.margins(0,0)
ax2.set_facecolor('#F4FBF3')
ax2.set_xticks(unstckd.index)
ax2.set_title('Kernzeit 11:30-19:30')
ax2.set_ylim(maxcalls)
p2 = ax2.bar(unstckd.index, kernzeit['B'], width=.8, color=nzb, bottom=kernzeit['H'], label="Ber")
p1 = ax2.bar(unstckd.index, kernzeit['H'], width=.8, color=nzh, label="Hag")

ax3=fig.add_subplot(133)
ax3.margins(0,0)
ax3.set_facecolor('#F4FBF3')
ax3.set_xticks(unstckd.index)
ax3.set_title('Nebenzeit 19:30-11:30')
ax3.set_ylim(maxcalls)
p2 = ax3.bar(unstckd.index, nebenzeit['B'], width=.8, color=nzb, bottom=nebenzeit['H'], label="Ber")
p1 = ax3.bar(unstckd.index, nebenzeit['H'], width=.8, color=nzh, label="Hag")

ax.legend()
ax2.legend()
ax3.legend()

In [ ]:
# Hier wird ein df erzeugt, der die einzelnen Zeiten als obere Ebene zusammenfasst, und darunter die Agenten
# namen der Spalten wie folgt:
# an=angenommene Anrufe, be=bearbeitete Anrufe, vl=verlorene Anrufe, tt=talktime absolut, acw=nacharbeit absolut
# zusätzlich, berechnet:ht=gesamtarbeitszeit (tt+acw), jeweils, aht att acw = durchschnitt (ht,tt,acw/be)

colfunx={'tt':'sum','acw':'sum','be':'sum','vl':'sum','an':'sum'}
monate=data.groupby(['mm','ag']).agg(colfunx)
monate['ht']=monate.tt+monate.acw
monate['att']=monate['tt']/monate.be
monate['aacw']=monate['acw']/monate.be
monate['aht']=monate['ht']/monate.be
ustck=monate.unstack()

alleagenten=monate.reset_index().groupby('mm')[['acw','tt','ht','be']].sum()
alleagenten['aht']=alleagenten['ht']/alleagenten['be']
alleagenten['att']=alleagenten['tt']/alleagenten['be']
alleagenten['aacw']=alleagenten['acw']/alleagenten['be']
alleagenten


ustck.loc[:,('be','alle')]=alleagenten['be']
ustck.loc[:,('aht','alle')]=alleagenten['aht']
ustck.loc[:,('att','alle')]=alleagenten['att']
ustck.loc[:,('aacw','alle')]=alleagenten['aacw']


In [ ]:
#ttmonatl(ustck,'gesinst')
#ustck.loc[7]['be']

In [ ]:
data.loc['2017-08'].head()

In [ ]:
stg=datetime.date(year=2017,month=5,day=5)
#data.loc[('2017-08','dehnsil'),:]
data.loc['2017-09','an'].sum()

**SUMME KW14 = SUMME AGENTENAUSWERTUNG STIMMT MIT SUMME HOTLINE ÜBEREIN**
**SUMME KW8: AGENTEN 19 WENIGER ALS HOTLINE. AGENTEN FEHLEN?**

In [ ]:
#data.be

In [ ]:
def ttmonatl(frame,agent):
    fig=plt.figure(figsize=(6,4))
    fig.suptitle('talk- und handlingtimes '+agent)
    aht="#003873"
    aacw="#EE0042"
    att="#899EB2"
    nzb="#C7798F"

    ax=fig.add_subplot(111)
    ax.margins(0,0)
    ax.set_xlabel('Monat')
    ax.set_ylabel('Minuten')
    ax.set_xticks(ustck.index)
    ax.set_ylim(0,10)
    ax.axhline(y=3.5,color=nzb,ls='--',alpha=0.6, label='3:30min')
    htime=ax.plot(frame.index,ustck['aht',agent],color=aht,label='aht')
    ttime=ax.plot(frame.index,ustck['att',agent],color=att,label='att')
    wtime=ax.plot(frame.index,ustck['aacw',agent],color=aacw,label='aacw')
    #for mon in htime:
    #    xpos=mon.get_xdata()
    #    heig=mon.get_ydata()
    #    for coord in xpos:
    #        calls=frame['be',agent].values[coord-2]
    #        note=ax.text(coord,ax.get_ylim()[1], s=str(int(calls)),ha='center')

    ax.legend()
    
    #### only for showing the y values
    print(ustck['aht',agent])

In [ ]:
ttmonatl(ustck,'alle')

In [ ]:
ustck.head()

In [ ]:
data.head()